In [ ]:
import sys
!{sys.executable} -m pip install --quiet langchain-experimental langchain-community langchain-core langchain-chroma langchain_openai langchain_text_splitters

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import getpass
import os
import urllib

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
tao_filename = 'tao.txt'
gita_filename = 'gita.txt'
gospels_filename = 'gospels.txt'

In [ ]:
def download_gospels():
    urllib.request.urlretrieve('https://www.gutenberg.org/cache/epub/10/pg10.txt', 'bible.txt')
    with open('bible.txt', 'r') as f:
        bible = f.read()
    start = bible.find('Saint Matthew', bible.find('Saint Matthew') + 1)
    end = bible.find('Acts of the Apostles', bible.find('Acts of the Apostles') + 1)
    gospels = bible[start:end]
    with open(gospels_filename, 'w') as f:
        f.write(gospels)

In [ ]:
if not (tao_filename in os.listdir() and gospels_filename in os.listdir() and gita_filename in os.listdir()):
    print("downloading texts and supporting texts")
    download_gospels()
    urllib.request.urlretrieve('https://www.gutenberg.org/cache/epub/2388/pg2388.txt', gita_filename)
    urllib.request.urlretrieve('https://www.gutenberg.org/cache/epub/216/pg216.txt', tao_filename)
else:
    print("using cached texts")


In [ ]:
def get_loader(filename, store, threshold):
    loader = TextLoader(filename)
    embeddings = OpenAIEmbeddings()
    documents = loader.load()[0].page_content
    text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="percentile", breakpoint_threshold_amount= threshold)
    
    texts = text_splitter.create_documents([documents], [{"source": filename}])
    print(len(texts))
    store.add_documents(texts)
    retriever = store.as_retriever(search_kwargs={"k": 1, "filter":{"source":filename}}) 
    return retriever, texts

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma("langchain_store", embeddings)

tao_retriever, tao_texts = get_loader(tao_filename, vectorstore, 30)
gita_retriever, gita_texts = get_loader(gita_filename, vectorstore, 30)
gospels_retriever, gospels_texts = get_loader(gospels_filename, vectorstore, 30)

In [ ]:
i = 0

In [ ]:
chunk = gita_texts[i]
content = chunk.page_content
print("-----------------")
print("gita:")
print(content)
print("\n")
print("tao:")
print(tao_retriever.invoke(content)[0].page_content)
print("\n")
print("gospel:")
print(gospels_retriever.invoke(content)[0].page_content)
print("-----------------")
print("\n\n\n")
i += 1